In [ ]:
pip install pytesseract

In [30]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import nltk
import pandas as pd
import re
from PIL import Image, ImageFilter
from PIL import Image, ImageEnhance
import os
import glob
import pytesseract
from pytesseract import Output
from google.colab.patches import cv2_imshow
import csv

In [ ]:
pip install datefinder

In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,276 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [71]:

# Create an empty DataFrame
columns = ['image_path', 'date', 'amount']
df= pd.DataFrame(columns=columns)
csv_filename = "/content/drive/MyDrive/DSEP/Background_removal /Receipts_with_background/Total_values/total_values.csv"
# Check if the CSV file exists, if not, create it
if not os.path.exists(csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(columns)




In [ ]:
# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'  # Path might vary, check your environment


In [ ]:

def ocr(image_path):
# Open the image using PIL (Python Imaging Library)
  try:
      image = Image.open(image_path)
      image = image.filter(ImageFilter.EDGE_ENHANCE)
      enhancer = ImageEnhance.Brightness(image)
      image = enhancer.enhance(1.2)  # Increase brightness by 20%
      enhancer = ImageEnhance.Contrast(image)
      image = enhancer.enhance(1.5)  # Increase contrast by 20%
      enhancer = ImageEnhance.Color(image)
      image = enhancer.enhance(1.5)  # Increase saturation by 20%

      # Perform OCR to extract text from the entire image
      text = pytesseract.image_to_string(image)


  except Exception as e:
      print(f"Error opening {image_path}: {str(e)}")
  return text


In [65]:
def save_bounding_box(image_path):
    global df
    try:
        # Read image to extract text from image
        img = cv2.imread(image_path)
        #cv2_imshow(img)
        # Convert image to gray scale
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Converting gray image to binary image by Thresholding
        thresh_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

        # Configuring parameters for tesseract
        custom_config = r'--oem 3 --psm 6'

        # Get OCR output details from pytesseract
        ocr_output_details = pytesseract.image_to_data(thresh_img, output_type=Output.DICT, config=custom_config, lang='eng')


        # Search for variations of "TOTAL" in the OCR output
        total_variations = ["Total", "Total due", "Due","TOTL"]

        # Search for the last occurrence of "Total" in the OCR output
        total_index = None
        for i in range(len(ocr_output_details['text']) - 1, -1, -1):
            word = ocr_output_details['text'][i]
            if any(variation in word for variation in total_variations):
                total_index = i
                break

        # Get the x and y coordinates of the "TOTAL" text box
        total_x = ocr_output_details['left'][total_index]
        total_y = ocr_output_details['top'][total_index]
        total_width = ocr_output_details['width'][total_index]
        total_height = ocr_output_details['height'][total_index]

        # Define the y range for selecting bounding boxes in the same horizontal area
        y_range = range(total_y - 10, total_y + total_height + 10)

        # Select bounding boxes with numbers in the same horizontal area
        selected_boxes = []
        for i, word in enumerate(ocr_output_details['text']):
            if any(char.isdigit() for char in word) and ocr_output_details['top'][i] in y_range:
                selected_boxes.append(i)

        # Find the largest selected box based on area
        largest_box_index = None
        largest_box_area = 0
        for box_index in selected_boxes:
            box_area = ocr_output_details['width'][box_index] * ocr_output_details['height'][box_index]
            if box_area > largest_box_area:
                largest_box_area = box_area
                largest_box_index = box_index

        # Draw bounding box around the largest selected box
        if largest_box_index is not None:
            x = ocr_output_details['left'][largest_box_index]
            y = ocr_output_details['top'][largest_box_index]
            w = ocr_output_details['width'][largest_box_index]
            h = ocr_output_details['height'][largest_box_index]
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Extract the text within the bounding box
            box_text = ocr_output_details['text'][largest_box_index]

            # Use regex to extract numerical values (including decimals)
            numerical_values = re.findall(r'\d+\.\d+|\d+', box_text)

            if numerical_values:
                values = [float(val) for val in numerical_values]  # Convert to floats
                print("Extracted Values:", values)
                new_data = pd.DataFrame({'image_path': [image_path], 'amount': [values]})
                df = pd.concat([df, new_data], ignore_index=True)
            else:
                print("No numerical value found within the bounding box")
        else:
            print("No bounding box found")



    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")




In [ ]:
import os

# Change the current working directory to '/content/drive'
drive_directory = "/content/drive/MyDrive/DSEP/Background_removal /Receipts_with_background/Scanned"
os.chdir(drive_directory)



In [ ]:
# Get the current working directory
folder_path = os.getcwd()

# List image file extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']

# List image files in the directory
image_files = [f for f in os.listdir(folder_path) if any(f.lower().endswith(ext) for ext in image_extensions)]



In [74]:
df

,image_path,date,amount
0,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[179.94]
1,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[38.86]
2,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[64.43]
3,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[69.79]
4,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[58.0]
5,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[44.35]
6,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[50.0]
7,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[143.0]
8,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[28.04]
9,/content/drive/MyDrive/DSEP/Background_removal...,NaN,[75.0]


In [75]:
  # index=False to exclude index column in the CSV


In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
# Process each image file
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    #print(ocr(image_path))
    save_bounding_box(image_path)
df.to_csv(csv_filename, index=False)

Error processing /content/drive/MyDrive/DSEP/Background_removal /Receipts_with_background/Scanned/1035-receipt.jpg: list indices must be integers or slices, not NoneType
Extracted Values: [179.94]
Extracted Values: [38.86]
Extracted Values: [64.43]
Extracted Values: [69.79]
Extracted Values: [58.0]
Error processing /content/drive/MyDrive/DSEP/Background_removal /Receipts_with_background/Scanned/1024-receipt.jpg: list indices must be integers or slices, not NoneType
Error processing /content/drive/MyDrive/DSEP/Background_removal /Receipts_with_background/Scanned/1058-receipt.jpg: list indices must be integers or slices, not NoneType
Error processing /content/drive/MyDrive/DSEP/Background_removal /Receipts_with_background/Scanned/1002-receipt.jpg: list indices must be integers or slices, not NoneType
Extracted Values: [44.35]
Extracted Values: [50.0]
Extracted Values: [143.0]
Extracted Values: [28.04]
Error processing /content/drive/MyDrive/DSEP/Background_removal /Receipts_with_backgrou